In [1]:
from instance_runner import InstanceRunner

In [2]:
parameters = {
    "threads": 4,
    "tempMin": 0.01,
    "tempMax": 2.25,
    "constructionBias": 3.0,
    "MKL": 400,
    "PTL": 2000,
    "tempD": 4,
    "upType": 1,
    "tempUpdate": 3,
    "movementType": 3
}

In [3]:
ir = InstanceRunner(
    parameters=parameters,
    root_path="/home/pedrowsl/cbm",
    instance_path="/home/pedrowsl/cbm/instances/h5",
    header_path="/home/pedrowsl/cbm/src/CBMProblem.hpp",
    output_path="/home/pedrowsl/cbm/output/a1_out.txt"
)
output, elapsed = ir.run()

Running: ./main_prd --filePath=/home/pedrowsl/cbm/instances/h5 --threads=4 --tempMin=0.01 --tempMax=2.25 --constructionBias=3.0 --MKL=400 --PTL=2000 --tempD=4 --upType=1 --tempUpdate=3 --movementType=3


In [4]:
output

'Solution: [152, 548, 796, 442, 410, 535, 728, 954, 29, 844, 339, 433, 907, 28, 785, 484, 528, 285, 154, 541, 975, 663, 847, 75, 400, 88, 257, 706, 498, 953, 76, 435, 132, 8, 853, 19, 7, 54, 342, 254, 929, 26, 901, 655, 872, 139, 832, 977, 885, 44, 963, 33, 830, 446, 809, 309, 793, 264, 380, 881, 603, 179, 789, 166, 776, 379, 239, 378, 738, 401, 16, 987, 784, 841, 883, 46, 91, 637, 892, 752, 753, 788, 792, 876, 231, 937, 417, 233, 245, 115, 544, 821, 622, 77, 194, 825, 397, 80, 270, 511, 540, 957, 525, 970, 128, 774, 389, 219, 40, 582, 513, 829, 260, 979, 313, 564, 448, 762, 24, 501, 802, 560, 512, 623, 848, 6, 67, 156, 735, 862, 79, 147, 687, 415, 915, 222, 997, 408, 807, 137, 781, 190, 837, 846, 760, 134, 890, 173, 581, 596, 158, 536, 660, 4, 602, 366, 712, 888, 903, 226, 998, 607, 74, 992, 742, 649, 452, 683, 958, 321, 279, 131, 208, 838, 57, 298, 895, 437, 48, 990, 492, 306, 648, 754, 626, 310, 676, 375, 191, 966, 424, 619, 875, 122, 692, 803, 117, 483, 610, 869, 277, 323, 49, 945,